# Inspect synthea dataset

In [1]:
from pathlib import Path
import pandas as pd
from dotenv import load_dotenv
import json
assert(load_dotenv())



Load Synthea Data

In [2]:
data_dir = Path("data/synthea")
dataframes = {}

for csv_file in data_dir.glob("*.csv"):
    key = csv_file.stem  # filename without .csv
    df = pd.read_csv(csv_file, header=0)
    dataframes[key] = df

# write human readable jsons for each dataframe to data/synthea/sample_structure.md
with open("data/synthea/sample_structure.md", "w", encoding="utf-8") as f:
    for key, df in dataframes.items():
        if df.empty:
            continue
        f.write(f"## {key}\n\n")
        f.write(f"```json\n{json.dumps(df.iloc[0].to_dict(), indent=2, ensure_ascii=False)}\n```")
        f.write("\n\n")


for key, df in dataframes.items():
    print(f"{key}:\t{sorted(df.columns.tolist())}")


allergies:	['CODE', 'DESCRIPTION', 'ENCOUNTER', 'PATIENT', 'START', 'STOP']
careplans:	['CODE', 'DESCRIPTION', 'ENCOUNTER', 'Id', 'PATIENT', 'REASONCODE', 'REASONDESCRIPTION', 'START', 'STOP']
conditions:	['CODE', 'DESCRIPTION', 'ENCOUNTER', 'PATIENT', 'START', 'STOP']
devices:	['CODE', 'DESCRIPTION', 'ENCOUNTER', 'PATIENT', 'START', 'STOP', 'UDI']
encounters:	['BASE_ENCOUNTER_COST', 'CODE', 'DESCRIPTION', 'ENCOUNTERCLASS', 'Id', 'ORGANIZATION', 'PATIENT', 'PAYER', 'PAYER_COVERAGE', 'PROVIDER', 'REASONCODE', 'REASONDESCRIPTION', 'START', 'STOP', 'TOTAL_CLAIM_COST']
imaging_studies:	['BODYSITE_CODE', 'BODYSITE_DESCRIPTION', 'DATE', 'ENCOUNTER', 'Id', 'MODALITY_CODE', 'MODALITY_DESCRIPTION', 'PATIENT', 'SOP_CODE', 'SOP_DESCRIPTION']
immunizations:	['BASE_COST', 'CODE', 'DATE', 'DESCRIPTION', 'ENCOUNTER', 'PATIENT']
medications:	['BASE_COST', 'CODE', 'DESCRIPTION', 'DISPENSES', 'ENCOUNTER', 'PATIENT', 'PAYER', 'PAYER_COVERAGE', 'REASONCODE', 'REASONDESCRIPTION', 'START', 'STOP', 'TOTALCOS

In [3]:
# check if all patient ids occour in the allergies table
all_patient_ids = set(dataframes["patients"]["Id"])
allergy_patient_ids = set(dataframes["allergies"]["PATIENT"])
print(f"all allergy patient ids in patients table?: {allergy_patient_ids.issubset(all_patient_ids)}")
print(f"all patient ids in allergies table?: {all_patient_ids.issubset(allergy_patient_ids)}")
print(f"Percentage of patients that have an allergy?: {len(allergy_patient_ids) / len(all_patient_ids) * 100:.2f}%")


all allergy patient ids in patients table?: True
all patient ids in allergies table?: False
Percentage of patients that have an allergy?: 12.04%


In [4]:
# which tables have patient ids
patient_id_tables = [key for key, df in dataframes.items() if "PATIENT" in df.columns]
print(f"Tables with patient IDs: {patient_id_tables}")
print(f"tables with patient id and no encounter id: {[key for key, df in dataframes.items() if 'PATIENT' in df.columns and 'ENCOUNTER' not in df.columns]}")

# are there patients that are associated to multiple encounters?
encounter_patient_ids = dataframes["encounters"]["PATIENT"]
print(f"are there patients that are associated to multiple encounters?: {len(encounter_patient_ids) > len(set(encounter_patient_ids))}")

Tables with patient IDs: ['allergies', 'careplans', 'conditions', 'devices', 'encounters', 'imaging_studies', 'immunizations', 'medications', 'observations', 'payer_transitions', 'procedures', 'supplies']
tables with patient id and no encounter id: ['encounters', 'payer_transitions']
are there patients that are associated to multiple encounters?: True


In [5]:
patients = dataframes["patients"]
patient = patients.sample()
print(patient.to_dict())

{'Id': {221: '45166484-2cc4-40f0-9be5-b9435ce0e476'}, 'BIRTHDATE': {221: '1967-02-20'}, 'DEATHDATE': {221: nan}, 'SSN': {221: '999-53-6733'}, 'DRIVERS': {221: 'S99998526'}, 'PASSPORT': {221: 'X88005000X'}, 'PREFIX': {221: 'Mrs.'}, 'FIRST': {221: 'Katia630'}, 'LAST': {221: 'Thiel172'}, 'SUFFIX': {221: nan}, 'MAIDEN': {221: 'Kuhn96'}, 'MARITAL': {221: 'M'}, 'RACE': {221: 'white'}, 'ETHNICITY': {221: 'nonhispanic'}, 'GENDER': {221: 'F'}, 'BIRTHPLACE': {221: 'Chelmsford  Massachusetts  US'}, 'ADDRESS': {221: '818 Rau Fort'}, 'CITY': {221: 'Boston'}, 'STATE': {221: 'Massachusetts'}, 'COUNTY': {221: 'Suffolk County'}, 'ZIP': {221: 2115.0}, 'LAT': {221: 42.39633957470782}, 'LON': {221: -71.1113862046484}, 'HEALTHCARE_EXPENSES': {221: 1224055.92}, 'HEALTHCARE_COVERAGE': {221: 5526.959999999999}}


In [6]:
# print 5 sample values for each column in encounters
encounter_df = dataframes["encounters"]

print("Encounters Table - Sample Values:\n")
print("=" * 80)

for col in encounter_df.columns:
    print(f"\n{col}:")
    samples = encounter_df[col].head(10).tolist()
    for i, value in enumerate(samples, 1):
        print(f"  {i}. {value}")


Encounters Table - Sample Values:


Id:
  1. d0c40d10-8d87-447e-836e-99d26ad52ea5
  2. e88bc3a9-007c-405e-aabc-792a38f4aa2b
  3. 8f104aa7-4ca9-4473-885a-bba2437df588
  4. b85c339a-6076-43ed-b9d0-9cf013dec49d
  5. dae2b7cb-1316-4b78-954f-fa610a6c6d0e
  6. 1e0d6b0e-1711-4a25-99f9-b1c700c9b260
  7. 6aa37300-d1b4-48e7-a2f8-5e0f70f48f38
  8. 9d35ec9f-352a-4629-92ef-38eae38437e7
  9. ae7555a9-eaff-4c09-98a7-21bc6ed1b1fd
  10. 7253a9f9-6f6d-429a-926a-7b1d424eae3f

START:
  1. 2010-01-23T17:45:28Z
  2. 2012-01-23T17:45:28Z
  3. 2001-05-01T15:02:18Z
  4. 2011-07-28T15:02:18Z
  5. 2010-07-27T12:58:08Z
  6. 2010-05-05T00:26:23Z
  7. 2011-04-30T00:26:23Z
  8. 2011-08-10T00:26:23Z
  9. 2011-11-17T00:26:23Z
  10. 2012-04-24T00:26:23Z

STOP:
  1. 2010-01-23T18:10:28Z
  2. 2012-01-23T18:00:28Z
  3. 2001-05-01T15:17:18Z
  4. 2011-07-28T15:17:18Z
  5. 2010-07-27T13:28:08Z
  6. 2010-05-05T01:11:23Z
  7. 2011-04-30T01:03:23Z
  8. 2011-08-10T00:41:23Z
  9. 2011-11-17T00:41:23Z
  10. 2012-04-24T00:41:23Z

P

In [9]:
# list all tables that do have a colum "ENCOUNTER"
tables_with_encounter = [name for name, df in dataframes.items() if "ENCOUNTER" in df.columns]
print(tables_with_encounter)

['allergies', 'careplans', 'conditions', 'devices', 'imaging_studies', 'immunizations', 'medications', 'observations', 'procedures', 'supplies']
